# Analysis of student grades and HRV data
- db application_project_gaube on localhost
- 10 Students, 3 Terms (mid1, mid2, final)
- SDNN & nni_mean available as well as interbeat iterval data

**Questions:**
1. Which exams turned out best?
2. Which student performed the best?
3. Is there a difference in the HRV for the whole recording time and the time in which the exams are written?
4. Is there a relation between grade and HRV?

## preparing

In [1]:
%pip install scipy
%pip install statsmodels

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.1.2 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.1.2 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import pandas as pd
import datetime as dt
import mysql.connector 
from scipy.stats import pearsonr
from statsmodels.formula.api import ols

In [3]:
db = mysql.connector.connect(
    host='localhost',
    user='root',
    password='',
    database='application_project_gaube'
)
cursor = db.cursor()

## getting answers

### 1. Which exams turned out best?

In [4]:
query_q1 = """
    SELECT
        e.id,
        e.term,
        AVG(md.grade) AS average_grade
    FROM
        master_data md
    JOIN exam e ON md.term_id = e.id
    JOIN dataset d ON md.student_id = d.id
    GROUP BY
        e.id, e.term;
    """

In [5]:
df_q1 = pd.read_sql(query_q1, db)

# dividing the final-grade by two because there are twice of the points possible
df_q1.loc[df_q1['term'] == 'final', 'average_grade'] = df_q1.loc[df_q1['term'] == 'final', 'average_grade'] / 2

C:\Users\49176\AppData\Local\Temp\ipykernel_1748\1651071942.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_q1 = pd.read_sql(query_q1, db)


In [6]:
df_q1

,id,term,average_grade
0,1,mid1,77.50
1,2,mid2,69.90
2,3,final,78.35


Answer: *The students perforemd in the final tests best*

### 2. Which student performed the best?

In [7]:
query_q2 = """
    SELECT
        d.student AS student,
        SUM(md.grade) AS total_points
    FROM
        master_data md
    JOIN exam e ON md.term_id = e.id
    JOIN dataset d ON md.student_id = d.id
    GROUP BY
        d.student;
"""

In [8]:
df_q2 = pd.read_sql(query_q2, db)

C:\Users\49176\AppData\Local\Temp\ipykernel_1748\2177194635.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_q2 = pd.read_sql(query_q2, db)


In [9]:
df_q2.sort_values(by='total_points', ascending=False)

,student,total_points
8,S8,364.0
3,S3,355.0
2,S2,347.0
0,S1,342.0
6,S6,310.0
4,S4,301.0
5,S5,301.0
1,S10,269.0
9,S9,245.0
7,S7,207.0


Answer: *Student S8 perforemd the best*

### 3. Is there a difference in the HRV for the whole recording time and the time in which the exams are written?
- the hrv is stored as moving_avarage values for the time of exam and as aggregated value for each recoding
- I use the ibi data instead of the 5min moving_avarage data for the mean and the sd of the ibi's during exams
- the timestamps of the term-periods are as followed:
    - mid1: (1539439200, 1539444600)
    - mid2: (1541862000, 1541867400)
    - final: (1544022000, 1544032800)

#### calculationg sdnn and nni_mean for term-period

In [10]:
query_q3a = """
    SELECT
        d.student,
        e.term,
        ibi.ibi_value
    FROM
        inter_beat_interval ibi
    JOIN
        dataset d ON ibi.student_id = d.id
    JOIN
        exam e ON ibi.term_id = e.id
    WHERE
        (ibi.term_id = 1 AND ibi.timestamp BETWEEN 1539439200 AND 1539444600)
        OR
        (ibi.term_id = 2 AND ibi.timestamp BETWEEN 1541862000 AND 1541867400)
        OR
        (ibi.term_id = 3 AND ibi.timestamp BETWEEN 1544022000 AND 1544032800)
"""

In [11]:
df_q3a = pd.read_sql(query_q3a, db)

C:\Users\49176\AppData\Local\Temp\ipykernel_1748\1691655406.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_q3a = pd.read_sql(query_q3a, db)


In [12]:
students_list = list(df_q3a.student.unique())
term_list = list(df_q3a.term.unique())

In [13]:
data_list = []
for stud in students_list:
    for examen in term_list:
        data_list.append({
            'student': stud,
            'term': examen,
            'nni_mean_examen': df_q3a.query('@stud == student and @examen == term').ibi_value.mean().round(2),
            'sdnn_examen': df_q3a.query('@stud == student and @examen == term').ibi_value.std().round(2)

    })

df_q3a= pd.DataFrame(data_list)

#### getting HRV for the whole recoding time

In [14]:
query_q3b = """
    SELECT
        d.student,
        e.term,
        md.nni_mean AS nni_mean_whole,
        md.sdnn AS sdnn_whole
    FROM
        master_data md
    JOIN
        dataset d ON md.student_id = d.id
    JOIN
        exam e ON md.term_id = e.id
"""

In [15]:
df_q3b = pd.read_sql(query_q3b, db)

C:\Users\49176\AppData\Local\Temp\ipykernel_1748\3555222216.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_q3b = pd.read_sql(query_q3b, db)


In [16]:
df_q3 = df_q3a.merge(df_q3b, on=['student', 'term'], how='inner')
df_q3['nni_mean_diff'] = df_q3['nni_mean_examen'] - df_q3['nni_mean_whole']
df_q3['sdnn_diff'] = df_q3['sdnn_examen'] - df_q3['sdnn_whole']

Note: *The HRV for the whole recording time is suptracted from the HRV during examen*

In [17]:
df_q3

,student,term,nni_mean_examen,sdnn_examen,nni_mean_whole,sdnn_whole,nni_mean_diff,sdnn_diff
0,S1,mid1,500.39,145.95,506.85,120.73,-6.46,25.22
1,S1,mid2,518.70,50.89,514.85,56.75,3.85,-5.86
2,S1,final,546.20,74.82,563.94,80.10,-17.74,-5.28
3,S10,mid1,560.73,216.00,512.53,154.06,48.20,61.94
4,S10,mid2,691.13,59.26,674.82,84.12,16.31,-24.86
5,S10,final,708.14,79.21,714.13,89.57,-5.99,-10.36
6,S2,mid1,654.44,244.50,532.73,137.46,121.71,107.04
7,S2,mid2,618.44,61.15,611.59,87.09,6.85,-25.94
8,S2,final,546.07,52.88,565.42,56.00,-19.35,-3.12
9,S3,mid1,677.15,228.68,554.05,139.93,123.10,88.75


In [18]:
print(f'nni_mean: {df_q3.nni_mean_diff.mean().round(2)}, sdnn: {df_q3.sdnn_diff.mean().round(2)}')

nni_mean: 17.16, sdnn: 5.8


Answer: *Over all students and all exams it seems that the HRV Time Domain measurements NNI mean and SDNN are higher during examen*

### 4. Is there a relation between grade and HRV?

In [19]:
query_q4 = """
    SELECT
        d.student,
        e.term,
        md.grade
    FROM
        master_data md
    JOIN
        dataset d ON md.student_id = d.id
    JOIN
        exam e ON md.term_id = e.id
"""

In [20]:
df_q4 = pd.read_sql(query_q4, db)

C:\Users\49176\AppData\Local\Temp\ipykernel_1748\3026019635.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_q4 = pd.read_sql(query_q4, db)


In [21]:
df_q4 = df_q4.merge(df_q3[['student', 'term', 'nni_mean_examen', 'sdnn_examen']], on=['student', 'term'], how='inner')

In [22]:
results_q4a = []
for examen in list(df_q4.term.unique()):
    subset = df_q4.query('term == @examen')
    for par in ['nni_mean_examen', 'sdnn_examen']:
        cor, p = pearsonr(subset.grade, subset[par])
        results_q4a.append({
            'term': examen,
            'parameter': par[:-7],
            'p': p,
            'cor': cor
        })
results_q4a = pd.DataFrame(results_q4a)

In [23]:
results_q4a

,term,parameter,p,cor
0,mid1,nni_mean,0.947609,-0.023964
1,mid1,sdnn,0.457679,-0.265942
2,mid2,nni_mean,0.311880,-0.356541
3,mid2,sdnn,0.950727,-0.022536
4,final,nni_mean,0.261435,-0.392852
5,final,sdnn,0.412567,-0.292248


Answer part1: *the correlations on term-level aren't significant, so they shouldn't be interpreted*

In [24]:
df_q4_manipulated = df_q4.copy()
df_q4_manipulated.loc[df_q4_manipulated['term'] == 'final', 'grade'] = df_q4_manipulated.loc[df_q4_manipulated['term'] == 'final', 'grade'] / 2

results_q4b = []
for par in ['nni_mean_examen', 'sdnn_examen']:
    cor, p = pearsonr(df_q4_manipulated.grade, df_q4_manipulated[par])
    results_q4b.append({
        'parameter': par[:-7],
        'p': p,
        'cor': cor
    })
results_q4b = pd.DataFrame(results_q4b)

In [25]:
results_q4b

,parameter,p,cor
0,nni_mean,0.068796,-0.33677
1,sdnn,0.829472,0.04105


Answer part2: over all exames there is an negativ correlation between the nni_mean and the grade. Accodring to the convention of Cohen the effect size is medium.

In [26]:
model = ols(f"grade ~ nni_mean_examen ", data=df_q4_manipulated)
result_model = model.fit()

In [27]:
result_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  grade   R-squared:                       0.113
Model:                            OLS   Adj. R-squared:                  0.082
Method:                 Least Squares   F-statistic:                     3.582
Date:                Sun, 29 Oct 2023   Prob (F-statistic):             0.0688
Time:                        19:18:15   Log-Likelihood:                -122.02
No. Observations:                  30   AIC:                             248.0
Df Residuals:                      28   BIC:                             250.8
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
===================================================================================
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept         106.2785     16.611      6.398      0.000      72.252     140.305
nni_mean_examen    -0.0500      0.026     -1.893      0.069      -0.104       0.004
==============================================================================
Omnibus:                        4.672   Durbin-Watson:                   2.431
Prob(Omnibus):                  0.097   Jarque-Bera (JB):                3.498
Skew:                          -0.829   Prob(JB):                        0.174
Kurtosis:                       3.222   Cond. No.                     3.91e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 3.91e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [28]:
df_q4.nni_mean_examen.std()

102.79878843152046

Answer part3: *The p-value for the regression weight of nni_mean isn't smaller then 0.05 but close to that. Subject to reservation, depending on this result it can be said that an increas of nni_mean by 0.05 worsens the perfomence of the students by one point. But depending on the high standarddeviasion in hrv-measures (102.8) the relation between hrv during examen and grade seems redundant*